In [4]:
import numpy as np
import sys
from river.datasets import synth
from river import datasets as rd
# import pandas as pd
import os
import torchvision
import numpy as np
import torch
import torchvision.transforms as transforms

class Interest():
    def __init__(self):
        self.current = [0, 1, 2]
        self.markov = self.generate_markov_matrix(10)
        self.interest_history = [[0, 1, 2]]

    def generate_markov_matrix(self, size=10):
        matrix = np.random.rand(size, size)
        np.fill_diagonal(matrix, matrix.diagonal() + 100)
        matrix = matrix / matrix.sum(axis=1)[:, np.newaxis]
        return matrix

    def one_hot_encode(self, index, size=10):
        vector = np.zeros(size)
        vector[index] = 1
        return vector

    def sample_new_interests(self):
        new_interests = []
        for interest in self.current:
            transition_probabilities = np.dot(self.one_hot_encode(interest), self.markov)
            new_interest = np.random.choice(range(self.markov.shape[1]), p=transition_probabilities)
            new_interests.append(new_interest)
        self.current = new_interests
        self.interest_history.append(new_interests)
        return new_interests


path = './data/cifar10'
transform = transforms.Compose([
    # transforms.ToPILImage(),
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

d1 = torchvision.datasets.CIFAR10(root=path, train=True, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10


In [5]:

class_indices = {i: np.where(np.array(d1.targets) == i)[0] for i in range(10)}

chunk_size = 500
num_classes = 10

from tqdm import tqdm
x = []
for i in tqdm(range(0, len(d1.data))):
    x.append(d1.__getitem__(i)[0])


new_y = []
new_x = []

for j in range(len(d1.data) // chunk_size):
    chunk_indices = []

    # Sample equally from each class's indices
    for class_label in class_indices:
        selected_indices = np.random.choice(class_indices[class_label], chunk_size // num_classes, replace=False)
        chunk_indices.extend(selected_indices)

        # Remove selected indices from the class_indices
        class_indices[class_label] = [idx for idx in class_indices[class_label] if idx not in selected_indices]
        if len(class_indices) == 0:
            print(j)
    # np.random.shuffle(chunk_indices)

    # Create the chunk
    y_chunk = [d1.targets[i] for i in chunk_indices]
    x_chunk = [x[i] for i in chunk_indices]

    new_y.append(y_chunk)
    new_x.append(np.stack(x_chunk))


y_chunks = np.array(new_y)
new_x = np.stack(new_x)
new_x = torch.from_numpy(new_x)
interest = Interest()

new_y = []
for y_chunk in y_chunks:
    # set label = 1 if label in interest.current
    chunk = []
    for i, y in enumerate(y_chunk):
        if y in interest.current:
            chunk.append(1)
        else:
            chunk.append(0)
    new_y.append(torch.tensor(chunk))
    interest.sample_new_interests()

new_y = torch.stack(new_y)

path = './data/cifar10/x.pth'
torch.save(new_x, path)
new_x = torch.load(path)
print(new_x.sum())

path = './data/cifar10/y.pth'
torch.save(new_y, path)
new_y = torch.load(path)
print(new_y.sum())



100%|██████████| 50000/50000 [00:06<00:00, 7636.13it/s]


tensor(-2879.6172)
tensor(12650)
